In [ ]:
from enum import Enum

import pandas as pd
from cuery import Prompt, ResponseModel, Task
from pydantic import Field

# Choices (enum)
Require LLM to respond with one of N _options_ (fixed categories).

In [2]:
class Role(Enum):
    PRINCIPAL = "PRINCIPAL"
    TEACHER = "TEACHER"
    STUDENT = "STUDENT"
    OTHER = "OTHER"


class UserDetail(ResponseModel):
    age: int
    name: str
    role: Role = Field(description="Correctly assign one of the predefined roles to the user.")


prompt = Prompt(
    messages=[
        {
            "role": "user",
            "content": "Please a create a synthetic user profile with age, name and role.",
        }
    ]
)

task = Task("user_profile", prompt=prompt, response=UserDetail)

response = await task()
print(response)
response.to_pandas()

[UserDetail(age=25, name='Alice', role=<Role.STUDENT: 'STUDENT'>)]


,age,name,role
0,25,Alice,Role.STUDENT


# Simple Multivalued fields
Require LLM to respond with a _list_ of values (unconstrained).

In [3]:
class Ingredients(ResponseModel):
    items: list[str] = Field(description="List of ingredients for the recipe.")


prompt = Prompt(
    messages=[
        {
            "role": "user",
            "content": "List the ingredients for the following dish: {{dish}}.",
        }
    ],
    required=["dish"],
)

context = [{"dish": "pasta bolognese"}, {"dish": "chocolate cake"}]
task = Task("dishes", prompt=prompt, response=Ingredients)
responses = await task(context=context)
print(responses)

Iterating context:   0%|          | 0/2 [00:00<?, ?it/s]

[Ingredients(items=['pasta', 'bolognese sauce', 'ground beef', 'onion', 'carrot', 'celery', 'garlic', 'tomato paste', 'red wine', 'beef broth', 'olive oil', 'salt', 'pepper', 'Parmesan cheese']), Ingredients(items=['chocolate', 'flour', 'sugar', 'eggs', 'butter', 'cocoa powder', 'baking powder', 'salt', 'milk', 'vanilla extract'])]


In [ ]:
# Maintain the original structure of the responses
responses.to_pandas(explode=False)

,dish,items
0,pasta bolognese,"[pasta, bolognese sauce, ground beef, onion, g..."
1,chocolate cake,"[flour, cocoa powder, sugar, eggs, butter, bak..."


In [ ]:
# Explode the list of ingredients into separate rows
responses.to_pandas(explode=True)

,dish,items
0,pasta bolognese,pasta
1,pasta bolognese,bolognese sauce
2,pasta bolognese,ground beef
3,pasta bolognese,onion
4,pasta bolognese,garlic
5,pasta bolognese,carrot
6,pasta bolognese,celery
7,pasta bolognese,red wine
8,pasta bolognese,tomato paste
9,pasta bolognese,beef broth


In [ ]:
# Convert to simple python records
responses.to_records(explode=False)

[{'dish': 'pasta bolognese',
  'items': ['pasta',
   'bolognese sauce',
   'ground beef',
   'onion',
   'garlic',
   'carrot',
   'celery',
   'red wine',
   'tomato paste',
   'beef broth',
   'salt',
   'pepper',
   'olive oil',
   'parmesan cheese']},
 {'dish': 'chocolate cake',
  'items': ['flour',
   'cocoa powder',
   'sugar',
   'eggs',
   'butter',
   'baking soda',
   'baking powder',
   'salt',
   'vanilla extract',
   'milk',
   'hot water']}]

# Nested models
Define a more complicated output structure by referencing another response model. 

In this case a list of certain length containing instances of pre-defined response model.

In [3]:
class Sector(ResponseModel):
    sector: str = Field(
        description="Human-readable title(!) of the industrical sector (in NAICS taxonomy)",
        min_length=10,
        max_length=150,
    )
    subsector: str = Field(
        description="Human-readable title(!) of the industrial SUBsector (in NAICS taxonomy)",
        min_length=5,
        max_length=150,
    )
    sector_automation_potential: int = Field(
        description="A score from 1 to 10 indicating the sector's potential for automation",
        ge=0,
        le=10,
    )


class Sectors(ResponseModel):
    sectors: list[Sector] = Field(
        description="A list of 1 to 5 NAIC industrial sectors with their AI automation potential",
        min_length=1,
        max_length=5,
    )


sectors_prompt = Prompt(
    messages=[
        {
            "role": "user",
            "content": "List some industrial sector in the country of {{country}} that have great AI automation potential.",
        }
    ],
    required=["country"],
)

context = [{"country": "Germany"}, {"country": "United States"}, {"country": "Japan"}]
sectors_task = Task("sectors", prompt=sectors_prompt, response=Sectors)
responses = await sectors_task(context=context)

Iterating context:   0%|          | 0/3 [00:00<?, ?it/s]

[05/29/25 12:27:47] WARNING  Encountered: 2 response parsing errors!                                    ]8;id=221965;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=686206;file:///Users/thomas/code/cuery/src/cuery/task.py#105\105]8;;\

In [4]:
responses.to_pandas(explode=True)

,country,sector,subsector,sector_automation_potential
0,Germany,Manufacturing,Automobile Manufacturing,8
1,Germany,Healthcare,Medical Devices,7
2,Germany,Information Technology,Software Development,9
3,United States,Manufacturing,Aerospace Product and Parts Manufacturing,8
4,United States,Health Care,Medical Laboratories,9
5,United States,Finance and Insurance,"Securities, Commodity Contracts, and Other Fin...",7
6,Japan,Manufacturing,Automotive Manufacturing,8
7,Japan,Technology,Electronics Manufacturing,9
8,Japan,Healthcare,Medical Devices,7
9,Japan,Information Technology,Software Development,8


# Chain tasks together
Run multiple tasks one after the other, collecting the results in a single DataFrame.

Keep in mind here that the names of inputs of one task must be the same as the names of outputs in the previous one.

Here we extract first some industrial sectors for each input country, and then some job roles within each sector.

In [5]:
# Re-uses "sectors" task from previous code cell (!)

from cuery import Chain


class Job(ResponseModel):
    job_role: str = Field(description="Name of the job role (job title, less than 50 characters)")
    job_description: str = Field(
        description="A short description of the job role (less than 200 characters)"
    )
    job_automation_potential: int = Field(
        description="A score from 1 to 10 indicating the job's potential for automation",
        ge=0,
        le=10,
    )


class Jobs(ResponseModel):
    jobs: list[Job] = Field(
        description=(
            "A list of jobs with their AI automation potential and reasons for that potential"
        ),
    )


jobs_prompt = Prompt(
    messages=[
        {
            "content": "List some job roles with great AI automation potential in the country of {{country}} and the sector '{{sector}}'"
        }
    ],
    required=["country", "sector"],
)

context = pd.DataFrame(
    {
        "country": ["Germany", "United States", "Japan"],
        "PIB": [4.0, 5.0, 3.5],
    }
)

jobs_task = Task("jobs", prompt=jobs_prompt, response=Jobs)
chain = Chain(sectors_task, jobs_task)
responses = await chain(context=context)

[05/29/25 12:27:50] INFO     [1/2] Running task 'Sectors'                                               ]8;id=170413;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=398104;file:///Users/thomas/code/cuery/src/cuery/task.py#189\189]8;;\

Iterating context:   0%|          | 0/3 [00:00<?, ?it/s]

[05/29/25 12:27:55] WARNING  Encountered: 1 response parsing errors!                                    ]8;id=164077;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=143673;file:///Users/thomas/code/cuery/src/cuery/task.py#105\105]8;;\

                    INFO     [2/2] Running task 'Jobs'                                                  ]8;id=490662;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=3405;file:///Users/thomas/code/cuery/src/cuery/task.py#189\189]8;;\

Iterating context:   0%|          | 0/9 [00:00<?, ?it/s]

In [7]:
responses

,country,sector,job_role,job_description,job_automation_potential
0,Germany,Manufacturing,Robotics Engineer,Design and develop robotic systems for automation,8
1,Germany,Manufacturing,Automation Technician,Install and maintain automated manufacturing e...,7
2,Germany,Manufacturing,Industrial Data Analyst,Analyzing data to improve manufacturing processes,6
3,Germany,Manufacturing,Process Automation Specialist,Implement and optimize automated processes in ...,9
4,Germany,Healthcare,Healthcare Data Analyst,Analyzing and interpreting data to improve hea...,8
5,Germany,Healthcare,Healthcare Robotics Engineer,Designing and developing robotics technology f...,9
6,Germany,Healthcare,Healthcare AI Specialist,Implementing AI solutions to enhance healthcar...,7
7,Germany,Finance and Insurance,Data Analyst,Analyzes data and generates insights for decis...,7
8,Germany,Finance and Insurance,Risk Manager,Identifies and mitigates financial risks withi...,6
9,Germany,Finance and Insurance,Actuary,Assesses and manages financial risks using sta...,8
